In [41]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import FAISS

from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain.prompts import PromptTemplate

import transformers as tm

In [28]:
from langchain.chains import RetrievalQA

In [39]:
import langchain
print(langchain.__version__)

0.3.7


## Read the PDFs from the folder

In [29]:
loader = PyPDFDirectoryLoader("./us_census")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)

final_documents[0]

Document(metadata={'source': 'us_census/acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under \nthe Continuous Enrollment Provision.2 The American \nRescue Plan (ARP) enhanced Marketplace premium \nsubsidies for those with incomes above 400 percent \nof the poverty level as well as for unemployed people.3\nIn addition to national policies, individual states and \nthe District of Columbia can affect h

In [30]:
len(final_documents)

316

## Embedding Using Huggingface

In [31]:
huggingface_embeddings=HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

In [32]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.30602050e-02 -1.45067042e-02 -2.10276116e-02  2.72682421e-02
  4.53646518e-02  5.28341159e-02 -2.53758654e-02  3.61304060e-02
 -9.08312201e-02 -2.77017578e-02  7.97397792e-02  6.42474890e-02
 -3.54004465e-02 -4.04244773e-02 -1.13771148e-02  4.45296168e-02
 -3.88552458e-03 -3.79054365e-03 -4.54509519e-02  2.67047361e-02
 -2.05681622e-02  2.87432298e-02 -2.41201390e-02 -3.69411707e-02
  1.92780644e-02  1.06195202e-02  3.21831694e-03  2.33250763e-03
 -4.29322459e-02 -1.64999247e-01  2.77007604e-03  2.68277302e-02
 -4.12894152e-02 -1.88446902e-02  1.58918500e-02  9.22327209e-03
 -2.00688597e-02  8.16561505e-02  3.89412865e-02  5.52223511e-02
 -3.69984433e-02  1.75319035e-02 -1.28966840e-02  2.80664448e-04
 -2.51580700e-02  4.59335512e-03 -2.39578579e-02 -5.76562667e-03
  6.02944149e-03 -3.61178108e-02  3.84415686e-02 -1.75467017e-03
  5.05656414e-02  6.02408648e-02  4.52067852e-02 -4.91435193e-02
  1.82053931e-02 -1.46667957e-02 -2.53130477e-02  3.18243504e-02
  5.15598804e-02 -9.32343

## VectorStore Creation

In [33]:

vectorstore=FAISS.from_documents(final_documents[:120], huggingface_embeddings)

## Query using Similarity Search

In [34]:
query="WHAT IS HEALTH INSURANCE COVERAGE?"

relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


### Create Retriever

In [35]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x284c44580> search_kwargs={'k': 3}


In [36]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv("HUGGINGFACEHUB_API_TOKEN")

The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [43]:
from langchain_huggingface.llms import HuggingFaceEndpoint, HuggingFacePipeline

In [47]:
hf = HuggingFacePipeline(
    model_id="mistralai/Mistral-7B-v0.1",
    # task="text-generation",
    model_kwargs={"temperature":0.1,"max_length":500},
)

query="What is the health insurance coverage?"

In [49]:
hf.invoke(query)

TypeError: 'NoneType' object is not callable

In [48]:
hf, query

(HuggingFacePipeline(model_id='mistralai/Mistral-7B-v0.1', model_kwargs={'temperature': 0.1, 'max_length': 500}),
 'What is the health insurance coverage?')

In [16]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}
)
query="What is the health insurance coverage?"
hf.invoke(query)

BadRequestError: (Request ID: T6jDW3TVU9k9_N8jlE5Ow)

Bad request:
Authorization header is correct, but the token seems invalid

# Hugging Face models can be run locally through the HuggingFacePipeline class.

In [40]:

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-v0.1.
401 Client Error. (Request ID: Root=1-6733271a-07d3a90055c1db5771781d6c;267c7f6c-971f-4d54-bdd1-91114675fd6c)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [ ]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [ ]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [ ]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])